# Ejercicio Educación
Por: Laura Vanessa Otálora Ramírez

- En este Notebook, vamos a continuar con nuestro ejercicio de educación. En el Notebook [Limpieza de Datos](./Limpieza_Datos_Educacion.ipynb), se realizó la limpieza de datos respectiva que vamos a utilizar para nuestro ejercicio.


In [ ]:
## cargamos nuestras librerias
import pandas as pd
import sqlite3
pd.set_option('display.max_columns', None) ## muestra todos los datos

### Data Frames a utilizar

In [2]:
edu = pd.read_csv('../Datos/Educacion_Limpio.csv')
pob = pd.read_csv('../Datos/Poblacion_Total.csv')

In [ ]:
conn = sqlite3.connect('../Datos/educacion.db')
cursor = conn.cursor()

In [4]:
def ejecutar_sql(query, conexion=conn):
  df = pd.read_sql_query(query, conexion)
  display(df)

In [5]:
# guardar como tablas SQL
edu.to_sql('educacion', conn, if_exists='replace', index=False)
pob.to_sql('poblacion', conn, if_exists='replace', index=False)

34782

### Verificar Tipos de Datos

Aunque esto lo verificamos en la limpieza de datos, es importante comprobar. 

In [6]:
ejecutar_sql("PRAGMA table_info(educacion);") # Para educación

,cid,name,type,notnull,dflt_value,pk
0,0,id_educacion,INTEGER,0,None,0
1,1,anio,INTEGER,0,None,0
2,2,codigo_municipio,INTEGER,0,None,0
3,3,nombre_municipio,TEXT,0,None,0
4,4,codigo_departamento,INTEGER,0,None,0
5,5,nombre_departamento,TEXT,0,None,0
6,6,poblacion_5_16,REAL,0,None,0
7,7,total_matriculados,REAL,0,None,0
8,8,tasa_matriculacion_5_16,REAL,0,None,0
9,9,cobertura_neta_total,REAL,0,None,0


In [7]:
ejecutar_sql("PRAGMA table_info(poblacion);") # Para población

,cid,name,type,notnull,dflt_value,pk
0,0,codigo_departamento,INTEGER,0,None,0
1,1,departamento,TEXT,0,None,0
2,2,codigo_municipio,INTEGER,0,None,0
3,3,municipio,TEXT,0,None,0
4,4,anio,INTEGER,0,None,0
5,5,poblacion_total,REAL,0,None,0


### Crear Tabla con "educacion_completa"

En esta tabla, donde tenemos los datos necesarios para construir el modelo. (Se añadio población total)

In [8]:
educacion_final = """
CREATE TABLE educacion_completa AS
SELECT 
    e.*, 
    p.poblacion_total
FROM 
    educacion e
LEFT JOIN 
    poblacion p
ON 
    e.codigo_municipio = p.codigo_municipio
    AND e.anio = p.anio
"""

cursor.execute(educacion_final)
conn.commit()  # guardar cambios

In [9]:
ejecutar_sql("SELECT name FROM sqlite_master WHERE type='table';") ## Ver todas las tablas de la bodega de datos. 

,name
0,educacion
1,poblacion
2,educacion_completa


### Dimensiones 
- Es importante tener en cuenta que las dimensiones contiene atributos (quien, dónde, cuándo)

In [10]:
dim_departamento = """
CREATE TABLE dim_departamento AS
SELECT DISTINCT
    codigo_departamento,
    nombre_departamento
FROM
    educacion_completa
ORDER BY codigo_departamento;
"""
cursor.execute(dim_departamento)
conn.commit()

In [11]:
ejecutar_sql("SELECT * FROM dim_departamento LIMIT 10;")

,codigo_departamento,nombre_departamento
0,5,Antioquia
1,8,Atlántico
2,11,"Bogotá, D.C."
3,11,Bogotá D.C.
4,13,Bolívar
5,15,Boyacá
6,17,Caldas
7,18,Caquetá
8,19,Cauca
9,20,Cesar


In [12]:
dim_municipio = """
CREATE TABLE dim_municipio AS
SELECT DISTINCT 
    codigo_municipio AS id_municipio,
    nombre_municipio,
    codigo_departamento AS id_departamento
FROM 
    educacion_completa
ORDER BY codigo_municipio;
"""

cursor.execute(dim_municipio)
conn.commit()

In [13]:
ejecutar_sql("SELECT * FROM dim_municipio LIMIT 10;")

,id_municipio,nombre_municipio,id_departamento
0,5001,Medellín,5
1,5002,Abejorral,5
2,5004,Abriaquí,5
3,5021,Alejandría,5
4,5030,Amagá,5
5,5031,Amalfi,5
6,5034,Andes,5
7,5036,Angelópolis,5
8,5038,Angostura,5
9,5040,Anorí,5


## Hechos
- Las tablas de hecho contiene los datos numericos. 

In [14]:
tabla_hechos = """
CREATE TABLE hechos_educacion AS
SELECT

    e.id_educacion,
    e.codigo_municipio,
    e.codigo_departamento,
    e.anio,
    
    e.total_matriculados,
    e.poblacion_total,
    
    e.tasa_matriculacion_5_16,
    e.cobertura_neta_total,
    e.cobertura_neta_transicion,
    e.cobertura_neta_primaria,
    e.cobertura_neta_secundaria,
    e.cobertura_neta_media,
    
    e.cobertura_bruta_total,
    e.cobertura_bruta_transicion,
    e.cobertura_bruta_primaria,
    e.cobertura_bruta_secundaria,
    e.cobertura_bruta_media,
    
    e.tasa_aprobacion_total,
    e.tasa_aprobacion_transicion,
    e.tasa_aprobacion_primaria,
    e.tasa_aprobacion_secundaria,
    e.tasa_aprobacion_media,
    
    e.tasa_desercion_total,
    e.tasa_desercion_primaria,
    e.tasa_desercion_secundaria,
    
    e.tasa_reprobacion_total,
    e.tasa_reprobacion_transicion,
    e.tasa_reprobacion_primaria,
    e.tasa_reprobacion_secundaria,
    e.tasa_reprobacion_media,
    
    e.tasa_repitencia_total,
    e.tasa_repitencia_transicion,
    e.tasa_repitencia_primaria,
    e.tasa_repitencia_secundaria,
    e.tasa_repitencia_media

FROM educacion_completa e;
"""
cursor.execute(tabla_hechos)
conn.commit()

In [15]:
ejecutar_sql("SELECT * FROM hechos_educacion LIMIT 10;")

,id_educacion,codigo_municipio,codigo_departamento,anio,total_matriculados,poblacion_total,tasa_matriculacion_5_16,cobertura_neta_total,cobertura_neta_transicion,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta_total,cobertura_bruta_transicion,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,tasa_aprobacion_total,tasa_aprobacion_transicion,tasa_aprobacion_primaria,tasa_aprobacion_secundaria,tasa_aprobacion_media,tasa_desercion_total,tasa_desercion_primaria,tasa_desercion_secundaria,tasa_reprobacion_total,tasa_reprobacion_transicion,tasa_reprobacion_primaria,tasa_reprobacion_secundaria,tasa_reprobacion_media,tasa_repitencia_total,tasa_repitencia_transicion,tasa_repitencia_primaria,tasa_repitencia_secundaria,tasa_repitencia_media
0,1,5004,5,2023,314.9786,2841.0,0.6262,0.6262,0.4419,0.6333,0.5153,0.4023,0.6680,0.5814,0.7286,0.6687,0.5632,0.9226,0.0000,0.9673,0.8349,0.9388,0.0119,0.0131,0.0000,0.0655,0.0000,0.0196,0.1651,0.0204,0.0952,0.0000,0.1046,0.1376,0.0204
1,2,95025,95,2023,2364.1226,16984.0,0.5327,0.5327,0.3391,0.4889,0.4490,0.2130,0.6298,0.5420,0.6519,0.6960,0.4854,0.8767,0.0000,0.8790,0.8450,0.9298,0.0556,0.0499,0.0611,0.0678,0.0000,0.0711,0.0939,0.0175,0.0934,0.0695,0.1184,0.0848,0.0316
2,3,95200,95,2023,654.9528,7947.0,0.3252,0.3252,0.1758,0.2533,0.2643,0.1075,0.3858,0.3636,0.3728,0.4610,0.2616,0.8268,0.0333,0.8464,0.7951,0.8750,0.0785,0.0843,0.0636,0.0947,0.0333,0.0693,0.1413,0.0781,0.0865,0.0667,0.0904,0.1025,0.0154
3,4,97001,97,2023,6544.3602,34037.0,0.5957,0.5957,0.4276,0.5595,0.4351,0.1706,0.7065,0.6490,0.7696,0.7292,0.5312,0.9071,0.0057,0.9412,0.8491,0.8993,0.0395,0.0184,0.0677,0.0534,0.0057,0.0404,0.0833,0.0460,0.1618,0.0775,0.2104,0.1384,0.0718
4,5,97161,97,2023,629.9640,3497.0,0.5130,0.5130,0.7632,0.5229,0.3371,0.1194,0.5554,0.9211,0.6521,0.5112,0.2736,0.8240,0.0000,0.8963,0.6900,0.7818,0.0836,0.0305,0.1572,0.0924,0.0000,0.0732,0.1528,0.0727,0.0924,0.0286,0.0762,0.1485,0.0364
5,6,97511,97,2023,910.9862,4698.0,0.5429,0.5429,0.1544,0.5569,0.2491,0.0462,0.7139,0.3893,1.0030,0.6792,0.3003,0.9465,0.0000,0.9284,0.9735,0.9341,0.0535,0.0716,0.0265,0.0000,0.0000,0.0000,0.0000,0.0000,0.2321,0.2586,0.3030,0.1425,0.0659
6,7,97666,97,2023,183.9640,2543.0,0.2260,0.2260,0.1429,0.3498,0.0000,0.0000,0.2334,0.2381,0.5776,0.0000,0.0000,1.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2158,0.1333,0.2229,0.0000,0.0000
7,8,97777,97,2023,0.0000,802.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
8,9,97889,97,2023,268.9995,1200.0,0.4795,0.4795,0.6875,0.5219,0.2584,0.1304,0.5205,1.0000,0.6974,0.3636,0.2717,0.9555,0.0000,0.9811,0.9342,0.8000,0.0445,0.0189,0.0658,0.0000,0.0000,0.0000,0.0000,0.0000,0.1233,0.0313,0.1950,0.0395,0.0400
9,10,99001,99,2023,6707.1732,22218.0,1.4452,1.4452,1.1444,1.6278,0.7848,0.3346,1.5850,1.4545,2.2567,1.2255,0.7543,0.8093,0.0000,0.8092,0.7536,0.8789,0.0581,0.0612,0.0580,0.1326,0.0000,0.1296,0.1884,0.0881,0.1118,0.0082,0.1325,0.1045,0.0639


### Resolucion de preguntas

#### 1. Respecto a la población del municipio ¿Que porcentaje de escolaridad hay?
- Por esta pregunta, buscamos por fuentes externas la población total de cada municipio por año. 

In [16]:
consulta_1 = """
SELECT 
    codigo_municipio,
    anio,
    ROUND((total_matriculados / poblacion_total) * 100, 4) AS porcentaje_escolaridad
FROM hechos_educacion
WHERE poblacion_total IS NOT NULL AND total_matriculados IS NOT NULL
ORDER BY codigo_municipio, anio;
"""
ejecutar_sql(consulta_1)


,codigo_municipio,anio,porcentaje_escolaridad
0,5001,2011,18.9833
1,5001,2012,18.4735
2,5001,2013,17.8160
3,5001,2014,17.2922
4,5001,2015,16.8677
...,...,...,...
14577,99773,2019,13.5568
14578,99773,2020,13.5122
14579,99773,2021,0.0147
14580,99773,2022,15.3908


#### 2. ¿Cómo compararía el rendimiento educativo por municipios?
- Usariamos el indicador "tasa_aprobacion_total"

In [17]:
consulta_2 = """
SELECT 
    codigo_municipio,
    anio,
    tasa_aprobacion_total
FROM hechos_educacion
WHERE tasa_aprobacion_total IS NOT NULL
ORDER BY codigo_municipio, anio;
"""
ejecutar_sql(consulta_2)


,codigo_municipio,anio,tasa_aprobacion_total
0,5001,2011,0.9628
1,5001,2012,0.9246
2,5001,2013,0.9051
3,5001,2014,0.9233
4,5001,2015,0.8592
...,...,...,...
14577,99773,2019,0.7006
14578,99773,2020,0.8027
14579,99773,2021,0.7355
14580,99773,2022,0.7273


#### 3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?
- Indicador: cobertura neta. se determino el promedio de cobertura neta por departamento.

In [19]:
consulta_3 = """
SELECT 
    codigo_departamento,
    ROUND(AVG(cobertura_neta_total), 2) AS cobertura_promedio
FROM hechos_educacion
WHERE cobertura_neta_total IS NOT NULL
GROUP BY codigo_departamento
ORDER BY cobertura_promedio DESC;
"""
ejecutar_sql(consulta_3)


,codigo_departamento,cobertura_promedio
0,63,0.95
1,70,0.94
2,20,0.94
3,47,0.93
4,50,0.91
5,85,0.89
6,73,0.89
7,25,0.89
8,11,0.89
9,23,0.88
